<a href="https://colab.research.google.com/github/Leonhart8902/Scratch_Detection_Thesis/blob/main/Skripsi_19_1_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Library Import<h1/>
<p>
<b> Import all library that will be used for the program later in one cell. The editing of the library all will be done in the cell below.<b/>
<p/>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import copy
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def Data_aug(org_dir,aug_dir,aug_num):
  '''
  org_dir = Refer to the original directory for the images
  aug_dir = Refer to the directory where the augmented images will be saved later
  aug_num = number of augmented image produces per original images
  '''
  datagen = ImageDataGenerator(
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

  # specify the directory where the original images are located
  directory = org_dir

  # loop through the directory and apply data augmentation to each image
  for root, dirs, files in os.walk(directory):
      for file in files:
          if file.endswith(".jpg"):
              img = cv2.imread(os.path.join(root, file))
              x = img.reshape((1,) + img.shape)
              i = 0
              for batch in datagen.flow(x, batch_size=1, save_to_dir = aug_dir, save_prefix='', save_format='jpg'):
                  i += 1
                  if i > aug_num: # number of augmented images to generate per original image
                      break


<h2> Creating Dataset <h2/>
<p>
<b>The codes below are used to create the dataset from the NG and G images that has been taken before<b/>
<p/>

In [ ]:
def Create_dataset(Directory):
  # set the directory path where the NG and G folders are located
  directory = Directory

  # create an empty list to store the image data
  image_data = []

  # loop through the NG and Good folders and read in the images
  for folder in ['NG', 'Good']:
      for filename in os.listdir(os.path.join(directory, folder)):
          if filename.endswith('.jpg'):
              # read in the image
              img = cv2.imread(os.path.join(directory, folder, filename))
              # append the image data and the corresponding label to the list
              image_data.append((img, folder))
  return image_data

<h2> Make the Copy of Your Dataset for the Comparison <h2/>
<p> 
<b>This copy is important so if you make any change in your dataset it won't destruct the original dataset that maybe you will need it in the future<b/>
<p/>

In [ ]:
def copy_dataset(Data_copied):
  # make a copy of the original dataset
  original_data = copy.deepcopy(Data_copied)
  return original_data

<h2> Resizing Your Dataset </h2>
<p>
<b>It's important for you to resizing your dataset so it could give a faster performance for your training later<b/>
</p>

In [ ]:
def Data_resize(Dataset):
  # set the desired image size
  new_size = (200,200)

  # loop through the image data and resize each image
  for i, data in enumerate(Dataset): #Enumerate used so the dataset won't change
      img, label = data
      # resize the image using numpy array
      img = np.array(img)
      img = cv2.resize(img, new_size)
      Dataset[i] = (img, label)
  return Dataset

<h1> Data Split <h1> 
<p>
<b> The data needs to be split into training data and testing data. The separated data will be treated differently. <b>
<p>

In [ ]:
def Split_data(Dataset):
  # split the data into training and testing sets
  train_data, test_data = train_test_split(Dataset, test_size=0.2)
  return train_data, test_data

<h1> Gabor Filtering <h1>
<p> <b> To improve the texture of the image trained or tested, filtering is needed. In this case the usage of Gabor Filtering who can create a high spacial data is used to improved the surface texture of the images. <b> <p>

In [ ]:
def Gabor_filtering(img):
  kernel = cv2.getGaborKernel((24, 24), 2.0, 0, 10.0, 0.5, 0, ktype=cv2.CV_32F)
  images = cv2.imread(img)
  filtered_img = cv2.filter2D(images, cv2.CV_8UC3, kernel)

<h2> Data Showing <h2/>
<p>
<b>The dataset that has been made before shown by using matplotlib. This data presentation is needed to check the quality of resized dataset.<b/>
<p>

In [ ]:
def Data_show(Dataset):
  for i, data in enumerate(Dataset):
    # get the first image and label from the dataset
    img, label = data
    # display the image using matplotlib
    plt.imshow(img)
    plt.title(label)
    plt.show()

<h1> Convert Labels <h1/>
<p>
<b> To train the data in the model with numerical approach, we need to convert the label in the data from 'NG' and 'Good' into the numerical value. <b>
<p>
<P>
<B> This kind of conversion were done by using the LabelEncoder from ScikitLearn. The LabelEncoder will turn the desired label to a number instead of text. <b>
<p>
<p>
<b> The label encoded then will be turned into OneHotEncoding for a better performance with multi-levels class like this program did. To turn it into OneHotEncoding, it needs to_categorical<b/>
<p>

In [ ]:
def Labeling (Dataset): 
  # convert the labels to numerical values
  encoder = LabelEncoder()
  y = [i[1] for i in Dataset]
  encoder.fit(y)
  y = encoder.transform(y)
  #convert the label encoder to oneHotEncoding using to_categorical
  y = to_categorical(y)

<h1> Code test Running <h1>

In [ ]:
dataset = Create_dataset("/content/drive/MyDrive/Dataset")
original_dataset = copy_dataset(dataset)

In [ ]:
resized_data = Data_resize(dataset)
train_data, test_data = Split_data(resized_data)

None
